#Team
#Chigozie Kenneth Okafor 225983
#Md Khamar Uz Zama 226267
#Rajatha Nagaraja Rao 223758

##SOURCE

[Keras Tutorial](https://keras.io/examples/cifar10_cnn/)

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras

from keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

Using TensorFlow backend.


In [ ]:
cifar100 = keras.datasets.cifar100
cifar10 = keras.datasets.cifar10
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()



train_labels = train_labels.reshape((-1,))
test_labels = test_labels.reshape((-1,))

# first difference: data is not reshaped to 784 anymore, but 28x28x1
# note the 1 color channel!! this is important
data = tf.data.Dataset.from_tensor_slices(
    (train_images.reshape([-1, 32, 32, 3]).astype(np.float32) / 255, train_labels.astype(np.int32)))
data = data.shuffle(buffer_size=60000).batch(128).repeat()

test_data = tf.data.Dataset.from_tensor_slices(
    (test_images.reshape([-1, 32, 32, 3]).astype(np.float32) / 255, test_labels.astype(np.int32))).batch(10000)

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])
plt.show()

In [ ]:
train_steps = 3000

initializer = tf.keras.initializers.GlorotUniform()



def build_model(width, height, depth, classes):
# initialize the input shape and channels dimension to be
	# "channels last" ordering
	inputShape = (height, width, depth)
	chanDim = -1
	# build the model using Keras' Sequential API
	model = tf.keras.Sequential([
		# CONV => RELU => BN => POOL layer set
		Conv2D(16, (3, 3), padding="same", input_shape=inputShape, kernel_initializer=initializer),
		Activation("elu"),
		MaxPooling2D(pool_size=(2, 2)),
		# (CONV => RELU => BN) * 2 => POOL layer set
		Conv2D(32, (3, 3), padding="same"),
		Activation("elu"),

		Conv2D(32, (3, 3), padding="same",  kernel_initializer=initializer),
		Activation("elu"),
		MaxPooling2D(pool_size=(2, 2)),
		# (CONV => RELU => BN) * 3 => POOL layer set
		Conv2D(64, (3, 3), padding="same",  kernel_initializer=initializer),
		Activation("elu"),

		Conv2D(64, (3, 3), padding="same"),
		Activation("elu"),

		Conv2D(64, (3, 3), padding="same",  kernel_initializer=initializer),
		Activation("elu"),

		MaxPooling2D(pool_size=(2, 2)),
		# first (and only) set of FC => RELU layers
		Flatten(),
		Dense(256),
		Activation("elu"),

		# softmax classifier
		Dense(classes),
		
	])
	# return the built model to the calling function
	return model

opt = tf.optimizers.Adam()
# from_logits = True!! #neverforget
loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

Adam optimizer learns faster for this particular dataset.

In [ ]:
# this basically hasn't changed
model = build_model(32, 32, 3, 10)

for step, (img_batch, lbl_batch) in enumerate(data):

    if step > train_steps:
        break

    with tf.GradientTape() as tape:
        img_batch = tf.image.per_image_standardization(img_batch)
        logits = model(img_batch)
        xent = loss_fn(lbl_batch, logits)

    grads = tape.gradient(xent, model.trainable_variables)
      
    opt.apply_gradients(zip(grads, model.trainable_variables))

    
    if not step % 100:
        preds = tf.argmax(logits, axis=1, output_type=tf.int32)
        acc = tf.reduce_mean(tf.cast(tf.equal(preds, lbl_batch),
                             tf.float32))
        print("Loss: {} Accuracy: {}".format(xent, acc))

In [ ]:
# here's some evaluation magic ;) bonus: figure out how this works...
big_test_batch = next(iter(test_data))
test_img = tf.image.per_image_standardization(big_test_batch[0])
test_preds = tf.argmax(model(test_img), axis=1,
                       output_type=tf.int32)

test_labels = tf.reshape(big_test_batch[1], [-1])
acc = tf.reduce_mean(tf.cast(tf.equal(test_preds, test_labels),
                             tf.float32))
print(acc)

In [ ]:
#Change this to pick different layers in the sequential model ConvD 1, convD 2 as 0, 3, etc.... [0, 3, 5, 8, 9, 10, 12]
Index_layer=0


weights = model.layers[Index_layer].get_weights()[0][:,:,0,:]
plt.title("Filters in Convolution layers")
for i in range(1,16):

  plt.subplot(4,4,i)
  plt.imshow(weights[:,:,i],interpolation="nearest",cmap="gray")
# plt.title("Filters in Convolution layers")  
plt.show()

In [ ]:
#pick an image from test data
index = 0


import cv2
images = big_test_batch[0][index]
images = images.numpy()
img = np.squeeze(images)
plt.imshow(img, cmap='gray')




## apply a specific set of filter weights (like the one displayed above) to the test image


pick_filter = 8


fig=plt.figure(figsize=(15, 5))
fig.add_subplot(2, 1, 1)
plt.imshow(weights[:,:,pick_filter], cmap='gray')
c = cv2.filter2D(img, -1, weights[:,:,pick_filter])
fig.add_subplot(2, 2, 1)
plt.imshow(c, cmap='gray')
plt.show()

https://medium.com/octavian-ai/which-optimizer-and-learning-rate-should-i-use-for-deep-learning-5acb418f9b2 shows the optimal learning rates for different optimzers used on MNIST dataset; The learning rate was fixed to 0.0001 and the training duration to 3 epochs in this case, though.

The used loss function was Sparse Categorical Cross-Entropy in every case

*   Test accuracy  w. SGD ~ 82%
*   Test accuracy  w. Adagrad ~ 84%
*   Test accuracy  w. Adam ~ 90%
*   Test accuracy  w. RMSProp ~ 90%

Using the CIFAR10-dataset and the same general CNN-layout with the appropriate adjustments to the parameters e.g. channels and number of inputs, the accuracy values deterioriated markedly:

*   Test accuracy  w. SGD ~ 48%
*   Test accuracy  w. Adagrad ~ 32%
*   Test accuracy  w. Adam ~ 70%
*   Test accuracy  w. RMSProp ~ 69%
